In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os.path as op
import pickle

import tensorflow as tf
from tensorflow import keras
from keras.models import Model,Sequential,load_model
from keras.layers import Input, Embedding
from keras.layers import Dense, Bidirectional
from keras.layers.recurrent import LSTM
import keras.metrics as metrics
import itertools
from tensorflow.python.keras.utils.data_utils import Sequence
from decimal import Decimal
from keras import backend as K
from keras.layers import Conv1D,MaxPooling1D,Flatten,Dense

2022-04-20 10:25:09.670326: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-20 10:25:09.670351: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
inp=pd.read_csv("../PJ sensor.csv",usecols=[6,7,10,11])
out=pd.read_csv("../PJ sensor.csv",usecols=[2,3,4,5,8,9])

FileNotFoundError: [Errno 2] No such file or directory: '../PJ sensor.csv'

In [ ]:
inp.head(5)

,Temperature,Relative Humidity,date,Time
0,31,65,191213,175844
1,31,65,191213,175904
2,31,65,191213,175924
3,31,65,191213,175944
4,31,66,191213,180004


In [ ]:
out.head(5)

,NO2,O3,CO,SO2,PM2.5,PM10
0,-15.075377,35.714286,1766.233766,148.484848,11.229150,17.323938
1,-20.100503,41.208791,1733.766234,142.424242,9.413914,24.967407
2,-15.075377,43.956044,1711.038961,136.363636,10.350110,56.781536
3,0.000000,57.692308,1688.311688,139.393939,10.488765,30.371115
4,-20.100503,54.945055,1681.818182,130.303030,12.353048,188.026886


In [ ]:
inp=np.array(inp)
out=np.array(out)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import warnings
scaler_obj=MinMaxScaler()
X1=scaler_obj.fit_transform(inp)
Y1=scaler_obj.fit_transform(out)

warnings.filterwarnings(action='ignore', category=UserWarning)

In [ ]:
# Splitting Data into training and testing dataset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=42)

from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

#creating object of sgboostregressor
model1=MultiOutputRegressor(XGBRegressor(tree_method='gpu_hist', gpu_id=0,objective ='reg:squarederror',max_depth=9,cosample_bytree=.5,learning_rate=.1,min_child_weight=3,
                   n_estimators=450,subsample=.7))
#training the model
# model_fit1=model1.fit(x_train,y_train)


eval_set = [(x_train, y_train), (x_test, y_test)]
eval_metric = ["auc","error"]
model_fit1=model1.fit(x_train, y_train)
%time model_fit1
print("Model training is Done!!")

filename1 = 'xgboost.sav'
pickle.dump(model_fit1, open(filename1, 'wb'))

[09:39:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "cosample_bytree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:40:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "cosample_bytree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:40:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "cosample_bytree" } might not be use

In [ ]:
from sklearn.metrics import r2_score
from sklearn import metrics
train_sizes=['NO2','O3','NO','CO','PM1','PM2.5','PM10']

#finding out the r2 score
y_train_pred1=model1.predict(x_train)
r2_train1=r2_score(y_train,y_train_pred1,multioutput='variance_weighted')

y_test_pred1=model1.predict(x_test)
r2_test1=r2_score(y_test,y_test_pred1,multioutput='variance_weighted')

print('r2 score on train data '+str(r2_train1))
print('r2 score on test data '+ str(r2_test1))

xgboost_mae=metrics.mean_absolute_error(y_test, y_test_pred1)
xgboost_mse=metrics.mean_squared_error(y_test, y_test_pred1)
xgboost_rmse=np.sqrt(xgboost_mse)
print('Mean Absolute Error:',xgboost_mae)
print('Mean Squared Error:',xgboost_mse )
print('Root Mean Squared Error:',xgboost_rmse)
print(' \n')


r2 score on train data 0.804894044676069
r2 score on test data 0.7867491776691122
Mean Absolute Error: 0.006571193824432935
Mean Squared Error: 0.0001733531624915415
Root Mean Squared Error: 0.013166364816893898
 



In [ ]:
import pickle
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=42)
loaded_model_fit7 = pickle.load(open("xgboost.sav", 'rb'))
y_test_pred=loaded_model_fit7.predict(x_test)
print("Predicted :\n",y_test_pred)
print("\n")
r2_test=r2_score(y_test,y_test_pred,multioutput='variance_weighted')
print("R2 Score : ",r2_test)

Predicted :
 [[0.48340052 0.5991475  0.04695781 0.0085996  0.06914376 0.00457146]
 [0.4310623  0.57405657 0.02534223 0.00805853 0.04595666 0.00399989]
 [0.48590872 0.6038791  0.05655789 0.00966027 0.10579658 0.00713156]
 ...
 [0.40851507 0.550818   0.03021315 0.01014406 0.06170817 0.00457344]
 [0.40562367 0.56409824 0.03133747 0.01069754 0.07458849 0.00595892]
 [0.47769344 0.56437266 0.06263287 0.00888559 0.09093509 0.00729026]]


R2 Score :  0.7896089086526633


In [12]:
train_sizes=['NO2','O3','NO','CO','PM2.5','PM10']
for i in range(0,6):
    temp=r2_score(y_test[:,i],y_test_pred[:,i])
    print("R2 Score of "+train_sizes[i]+":",temp)

R2 Score of NO2: 0.8195703034769317
R2 Score of O3: 0.6531202620252131
R2 Score of NO: 0.9357851105806742
R2 Score of CO: 0.6349962548363777
R2 Score of PM2.5: 0.8655713228023709
R2 Score of PM10: 0.32998446478897325


In [10]:
train_sizes=['NO2','O3','NO','CO','PM2.5','PM10']
for i in range(0,6):
    temp=metrics.mean_squared_error(y_test[:,i],y_test_pred[:,i])
    print("MSE Value of "+train_sizes[i]+":",temp)

MSE Value of NO2: 0.0003454472029571804
MSE Value of O3: 0.0004640735209453167
MSE Value of NO: 1.2321136965961492e-05
MSE Value of CO: 1.1065137761087797e-06
MSE Value of PM2.5: 0.00018894389102202974
MSE Value of PM10: 1.3277543064390998e-05


In [13]:
train_sizes=['NO2','O3','NO','CO','PM2.5','PM10']
for i in range(0,6):
    temp=np.sqrt(metrics.mean_squared_error(y_test[:,i],y_test_pred[:,i]))
    print("RMSE Value of "+train_sizes[i]+":",temp)

RMSE Value of NO2: 0.018586210021335182
RMSE Value of O3: 0.021542365723042505
RMSE Value of NO: 0.003510147712840799
RMSE Value of CO: 0.0010519095855199628
RMSE Value of PM2.5: 0.013745686269591263
RMSE Value of PM10: 0.003643836311415621


In [14]:
train_sizes=['NO2','O3','NO','CO','PM2.5','PM10']
for i in range(0,6):
    temp=metrics.mean_absolute_error(y_test[:,i],y_test_pred[:,i])
    print("MAE Value of "+train_sizes[i]+":",temp)

MAE Value of NO2: 0.012658108520777174
MAE Value of O3: 0.0127259398499967
MAE Value of NO: 0.00195449265954016
MAE Value of CO: 0.0004553515460001868
MAE Value of PM2.5: 0.009424909611260931
MAE Value of PM10: 0.0020360532648404676


In [ ]:
#completed